### Naludaq Version
*Max Version*: `0.17.2`  
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `UPAC96`
    + Firmware: `v911` - `v911`

### Equipment Needed
+ Function Generator
+ Oscilloscope

## Example Code

`Give descriptions and/or reasoning of code`

This notebook is to demonstrate how to capture an event with Naludaq's software on the UPAC96 with software trigger.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
from collections import deque

import matplotlib.pyplot as plt
from logging import getLogger, DEBUG, StreamHandler, Formatter

from naludaq.board import Board, startup_board
from naludaq.controllers import get_board_controller
from naludaq.daq.workers.worker_serial_reader import SerialReader
from naludaq.communication import (
    ControlRegisters,
    DigitalRegisters,
    AnalogRegisters,
)

### Setup Logger

In [ ]:
def setup_logger(connection_level = DEBUG):
    """Setup a logger for Naludaq to print out debug information.
    
    Args:
        connection_level (int): Sets logging level for UART/FTDI connections.
            Warning: If set too low, VS Code may crash
    """
    logger = getLogger()
    logger.setLevel(DEBUG)

    # UART is very verbose, keep on info unless something is broken.
    # getLogger("naludaq.UART").setLevel(connection_level)
    # getLogger("naludaq.FTDI").setLevel(connection_level)

    handler = StreamHandler()
    handler.setFormatter(Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s"))
    logger.addHandler(handler)

    return logger

try:
    logger
except:
    logger = setup_logger()


### Board Creation and Startup

In [ ]:
MODEL = "hiper"
SERIAL = "EA8C7B"
BAUDRATE = 3_000_000
BOARD = Board(MODEL)
BOARD.get_ftdi_connection(serial_number=SERIAL, baud=BAUDRATE)
BOARD.params["installed_chips"] = [0,1]
BOARD.disconnect()

In [ ]:
with BOARD:
    startup_board(BOARD)

### Capture Data

Data is captured with a debugdaq, and events are stored to the DebugDaq's `output_buffer` property

In [ ]:
bc = get_board_controller(BOARD)
CR = ControlRegisters(BOARD)
DR = DigitalRegisters(BOARD)

In [ ]:
with BOARD:
    bc.reset_board()

In [ ]:
with BOARD:
    print(bc.read_identifier()["value"])

In [ ]:
def arm_trigger(board):
    CR.write("trig_count_reset", True)
    CR.write("trig_count_reset", False)


def set_maximum_triggers(board, trig_enable, value):
    """Sets the maximum amount of triggers.

    Args:
        trig_enable (int): the chip number
        value (int): the maximum number of triggers
    """
    CR.write("exttrig_en", trig_enable)
    CR.write("max_trig_count", value)

def set_maximum_triggers(board, trig_enable, value):
    """Sets the maximum amount of triggers.

    Args:
        trig_enable (int): the chip number
        value (int): the maximum number of triggers
    """
    CR.write("exttrig_en", trig_enable)
    CR.write("max_trig_count", value)


In [ ]:
from naludaq.controllers.readout import get_readout_controller
readwindow = {
    "windows": 6,
    "lookback": 12,
    "write_after_trig": 6
}

with BOARD:
    get_readout_controller(BOARD).set_read_window(**readwindow)
    CR.write("header_en", 1)
    CR.write("footer_en", 1)


In [ ]:
output = deque()
sread = SerialReader(BOARD.connection, output)
readout_settings = {
    "trig": "ext",
    "lb": "forced",
    "acq": "raw",
    "dig_head": False,
    "ped": "zero",
    "readoutEn": True,
    "singleEv": False,
}

# with BOARD:
#     for chip in BOARD.params["installed_chips"]:
#         set_maximum_triggers(BOARD, chip, 2)

# Opening the board with a context manager, ensures the connection is closed when we are done using it
with BOARD:
    DR.write('enabletestpatt', 1)
    sread.start()
    bc.start_readout(**readout_settings)
    time.sleep(0.1)

    bc.toggle_trigger()
    time.sleep(5)
    sread.stop()
    bc.stop_readout()

In [ ]:
lol = bytearray()
for pk in output:
    lol.extend(pk)

for i in range(10, len(lol), 2):
    print(f"{i}: {lol[i:i+2].hex()}")

In [ ]:
with BOARD:
    idl=CR.read('idle_det')['value']
    print(f"{idl=}, {idl=:#010b}, {idl=:#010x}")
    bl = CR.read('busy_locked')['value']
    print(f"{bl=}, {bl=:#010b}, {bl=:#010x}")